In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import geojson
import rasterio
import pygmsh
import firedrake
from firedrake import Constant, assemble, sqrt, inner, grad, avg, jump, dx, dS
import icepack
from icepack.constants import (
    ice_density as ρ_I, gravity as g, weertman_sliding_law as m
)

Load up the hand-digitized outline of the glacier, convert it into the input format for gmsh, generate the mesh using gmsh, and then load up the triangular mesh.

In [ ]:
outline_filename = "beardmore.geojson"
with open(outline_filename, "r") as outline_file:
    outline = geojson.load(outline_file)

geometry = icepack.meshing.collection_to_geo(outline)
with open("beardmore.geo", "w") as geo_file:
    geo_file.write(geometry.get_code())
    
command = "gmsh -2 -format msh2 -v 2 -o beardmore.msh beardmore.geo"
subprocess.run(command.split(" "))

mesh = firedrake.Mesh("beardmore.msh")

Fetch the Mosaic of Antarctica so that we can make pretty pictures.

In [ ]:
coords = np.array(list(geojson.utils.coords(outline)))
delta = 10e3
xmin, xmax = coords[:, 0].min() - delta, coords[:, 0].max() + delta
ymin, ymax = coords[:, 1].min() - delta, coords[:, 1].max() + delta

In [ ]:
image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    height, width = image_file.height, image_file.width
    transform = image_file.transform
    window = rasterio.windows.from_bounds(
        left=xmin,
        bottom=ymin,
        right=xmax,
        top=ymax,
        transform=transform,
    )
    image = image_file.read(indexes=1, window=window, masked=True)

In [ ]:
def subplots(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.set_aspect("equal")
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap="Greys_r",
        vmin=12e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax),
    )

    return fig, axes

Show the mesh on top of some imagery so we can make sure everything is where it should be.

In [ ]:
fig, axes = subplots()
kwargs = {
    "interior_kw": {"linewidth": 0.25},
    "boundary_kw": {"linewidth": 2},
}
firedrake.triplot(mesh, axes=axes, **kwargs)
axes.legend();

Make some function spaces -- here we're deciding to use piecewise quadratic basis functions in each triangle to represent fields defined on this mesh.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family="CG", degree=2)
V = firedrake.VectorFunctionSpace(mesh, family="CG", degree=2)

Start fetching some observational data sets and interpolating them to our finite element spaces.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness_filename = f"netcdf:{bedmachine_filename}:thickness"
with rasterio.open(thickness_filename, "r") as thickness_file:
    h = icepack.interpolate(thickness_file, Q)
    
surface_filename = f"netcdf:{bedmachine_filename}:surface"
with rasterio.open(surface_filename, "r") as surface_file:
    s = icepack.interpolate(surface_file, Q)

Plot the thickness and surface elevation.

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(s, axes=axes)
fig.colorbar(colors);

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(h, axes=axes)
fig.colorbar(colors);

Now fetch and interpolate the velocities.

In [ ]:
measures_filename = icepack.datasets.fetch_measures_antarctica()
with rasterio.open(f"netcdf:{measures_filename}:VX", "r") as vx_file, \
     rasterio.open(f"netcdf:{measures_filename}:VY", "r") as vy_file:
    u_obs = icepack.interpolate((vx_file, vy_file), V)

In [ ]:
log_norm = matplotlib.colors.LogNorm(vmin=1.0, vmax=400.0)

fig, axes = subplots()
streamlines = firedrake.streamplot(
    u_obs, norm=log_norm, axes=axes, resolution=2.5e3, seed=1729
)
fig.colorbar(streamlines);

Next we'll calculate a very smoothed-over driving stress.
We'll use this to guess at the friction coefficient.

In [ ]:
α = Constant(10e3)
τ = firedrake.Function(V)
τ_d = -ρ_I * g * h * grad(s)
misfit = 0.5 * inner(τ - τ_d, τ - τ_d) * dx
smoothness = 0.5 * α ** 2 * inner(grad(τ), grad(τ)) * dx
J = misfit + smoothness
F = firedrake.derivative(J, τ)
firedrake.solve(F == 0, τ)

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(τ, axes=axes)
fig.colorbar(colors);

We'll estimate that basal friction coefficient takes up half the driving stress and that the ice is at a uniform temperature of -13C.

In [ ]:
fraction = Constant(0.5)
expr = fraction * sqrt(inner(τ, τ)) / sqrt(inner(u_obs, u_obs)) ** (1 / m)
C = firedrake.interpolate(expr, Q)
area = Constant(assemble(Constant(1.0) * dx(mesh)))
C_0 = Constant(assemble(C * dx) / float(area))

In [ ]:
T = Constant(260.0)
A = icepack.rate_factor(T)

We'll modify how the basal friction is calculated so that we can easily estimate what the friction coefficient is.

In [ ]:
def friction(**kwargs):
    u = kwargs["velocity"]
    θ = kwargs["log_friction"]
    C = C_0 * firedrake.exp(θ)
    return icepack.models.friction.bed_friction(velocity=u, friction=C)

In [ ]:
model = icepack.models.IceStream(friction=friction)
opts = {
    "dirichlet_ids": [1, 2, 3, 4, 5, 6, 7],
    "diagnostic_solver_type": "petsc",
    "diagnostic_solver_parameters": {
        "snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts)

In [ ]:
θ = firedrake.Function(Q)

In [ ]:
u = solver.diagnostic_solve(
    velocity=u_obs,
    thickness=h,
    surface=s,
    fluidity=A,
    log_friction=θ,
)

The results are much faster than observations but not so far out there that we can't tweak them a bit.

In [ ]:
fig, axes = subplots()
streamlines = firedrake.streamplot(
    u, norm=log_norm, axes=axes, resolution=2.5e3, seed=1729
)
fig.colorbar(streamlines);

We need to do a bit of a dirty hack in order to use biharmonic regularization.
It involves some funny penalty terms involving the minimum angle between any two edges of the mesh.
The cell below calculates this minimum angle.

In [ ]:
from numpy.linalg import norm

coords = mesh.coordinates.dat.data_ro
cells = mesh.coordinates.cell_node_map().values

min_angle = np.inf
for cell in cells:
    for k in range(3):
        x, y, z = coords[np.roll(cell, k)]
        ζ, ξ = y - x, z - x
        angle = np.arccos(np.inner(ζ, ξ) / (norm(ζ) * norm(ξ)))
        min_angle = min(angle, min_angle)

I've explained a bit of the sorcery below on [my website](https://shapero.xyz/posts/nitsches-method/) but you probably shouldn't ever look at that.
You should just be able to copy-paste this and call it good.
The only thing you might need to modify is to increase $L$ if you want a smoother solution and decrease it if the fit to the observations isn't good enough.

In [ ]:
L = Constant(5e2)

def regularization(θ):
    dθ = grad(θ)
    ν = firedrake.FacetNormal(mesh)
    dθ_dν = inner(dθ, ν)
    
    degree = θ.ufl_element().degree()
    ℓ = avg(firedrake.CellSize(mesh))
    γ = Constant(
        4 * (degree - 1) * degree / (np.sin(min_angle) * np.tan(min_angle / 2))
    )

    internal = inner(grad(dθ), grad(dθ)) * dx
    edge_constraint = 2 * avg(inner(grad(dθ_dν), ν)) * jump(dθ_dν) * dS
    edge_penalty = γ / ℓ * jump(dθ_dν)**2 * dS
    weight = 0.5 * L**4 / area
    
    return weight * (internal + edge_constraint + edge_penalty)

Now let's try and see what the basal friction has to be.

In [ ]:
def simulation(θ):
    return solver.diagnostic_solve(
        velocity=u_obs,
        thickness=h,
        surface=s,
        fluidity=A,
        log_friction=θ,
    )

σx = Constant(5.0)
σy = Constant(5.0)
def loss_functional(u):
    δu = u - u_obs
    return 0.5 / area * ((δu[0] / σx)**2 + (δu[1] / σy)**2) * dx

In [ ]:
from icepack.statistics import StatisticsProblem, MaximumProbabilityEstimator

problem = StatisticsProblem(
    simulation=simulation,
    loss_functional=loss_functional,
    regularization=regularization,
    controls=θ,
)

In [ ]:
estimator = MaximumProbabilityEstimator(
    problem,
    gradient_tolerance=1e-5,
    step_tolerance=1e-3,
    max_iterations=50,
)
θ = estimator.solve()

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(θ, vmin=-3, vmax=+3, axes=axes)
fig.colorbar(colors);

The result does a pretty good job matching hte observed velocities.

In [ ]:
u = simulation(θ)

In [ ]:
fig, axes = subplots()
streamlines = firedrake.streamplot(
    u, norm=log_norm, axes=axes, resolution=2.5e3, seed=1729
)
fig.colorbar(streamlines);

In [ ]:
δu = firedrake.interpolate(sqrt(inner(u - u_obs, u - u_obs)), Q)

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(δu, vmin=0.0, vmax=20.0, axes=axes)
fig.colorbar(colors);

Save the results to disk; we'll resume the simulation in another notebook.

In [ ]:
C = firedrake.interpolate(C_0 * firedrake.exp(θ), Q)
filename = "modern_state.h5"
with firedrake.CheckpointFile(filename, "w") as chk:
    chk.save_function(C, name="friction")
    chk.save_function(u, name="velocity")

But a bit of analysis is in order.
Just because the velocity that we compute matches observations well doesn't mean that it gives a reasonable local dynamic mass balance.
The figure below shows $\nabla\cdot hu$, which reaches +/-20 m/yr throughout much of the trunk.
We can try to fix this if it's important.
It results in pretty big transients and that's why we'll need fine temporal resolution to initialize a spin-up from here.

In [ ]:
from firedrake import div

q = firedrake.project(div(h * u), Q)

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(q, vmin=-20, vmax=+20, cmap="RdBu", axes=axes)
fig.colorbar(colors);

To inform the later simulations, it's also worth looking at what the steady state accumulation rate would have to be to account for the difference between inflow and outflow.

In [ ]:
from firedrake import ds

outflow_ids = (4,)
inflow_ids = (2,)

ν = firedrake.FacetNormal(mesh)
outflow = assemble(h * inner(u, ν) * ds(outflow_ids))
inflow = -assemble(h * inner(u, ν) * ds(inflow_ids))
print(f"Inflow:                   {inflow / 1e9:5.3f} km³ / yr")
print(f"outflow:                  {outflow / 1e9:5.3f} km³ / yr")

acc = (outflow - inflow) / float(area)
print(f"Avg steady accumulation:  {acc:5.3f} m / yr")